In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Tabular Playground Series - Apr 2022 : LSTM VS GRU Baseline for Starter
</h1>
</div>

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/25226/logos/header.png?t=2021-01-27-17-34-31" alt="">

<img src="https://www.tutorialexample.com/wp-content/uploads/2020/07/LSTM-Vs-GRU-Network-Which-Has-better-Performance.png" alt="">

In this notebook, we are going to compare two basic RNN based models, LSTM and GRU by using user friendly Keras.

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Data Preprocessing
</h1>
</div>

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
labels_df = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")
test_df = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")
features = train_df.iloc[:, 3:].columns.tolist()

# setting seed for better comparison.
num_seed = 2
np.random.seed(num_seed)
tf.random.set_seed(num_seed)
print("tensorflow & numpy random seed : {}".format(num_seed) )

In [ ]:
sequences = [0, 1, 2, 8364, 15404]
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train_df[train_df.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()

#from ambrosm

In [ ]:
fig = px.imshow(train_df.iloc[:,3:].corr(), color_continuous_scale='RdBu_r', text_auto=True, aspect='auto')
fig.show()

In [ ]:
# preprocessing the data
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])

In [ ]:
label=labels_df['state']
train_arr = train_df[features].values
train_arr = train_arr.reshape(-1, 60, train_arr.shape[-1])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_arr, label, test_size=0.2, shuffle=True, stratify=label, random_state=2)

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
LSTM Model
</h1>
</div>

## LSTM
LSTM was proposed to solve the Long-Term Dependency problem in that the previous context does not extend to the back. If you look at the actual thesis, it is quite complex and is organized mathematically, but if you summarize it with a picture, it will look like this.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fza6wJ%2FbtqGcgIT7pe%2FDWkx3rT0x8lKwJhoR6bJL0%2Fimg.png" alt="">

#### LSTM has been proposed to increase the context transfer rate when the time step is long in RNN. 
The key is 'putting in a gate that determines how much information is forgotten and how much it leads directly to the next step'. The process is summarized by calculating the central sigma, tanh, etc.The good news for us here is that even if you do not know the exact structure of the LSTM Cell, you can easily call it and write it by using a deep learning framework such as Keras, rather than designing it yourself .

In [ ]:
def LSTM():
    inputs = keras.Input(shape=(60, train_arr.shape[-1]))
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(inputs)
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(256))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16)(x)
    outputs = layers.Dense(1, 'sigmoid')(x)

    model = keras.Model(inputs, outputs)
    
    model.summary()
    
    model_path = './LSTM.h5'
    checkpointer = ModelCheckpoint(
        filepath=model_path,
        monitor='val_auc',
        verbose=1,
        mode='max',
        save_best_only=True)

    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['AUC'])
    
    return model, checkpointer

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
GRU Model
</h1>
</div>

## GRU

<img src="https://miro.medium.com/max/1106/1*42nQ3Uuem54uTPPY0PZmyw.png" alt="">

GRU was created for a similar reason to LSTM, but it is a more simplified version of Time-Step Cell that composes LSTM. It is a model from a thesis published in 2014 by Professor Cho Kyung-hyeon of Korea, and it can be said that the calculation speed is faster without being greatly pushed out in terms of accuracy as it is 'simplified'. This can also be easily imported using Keras.



In [ ]:
def GRU():
    inputs = keras.Input(shape=(60, train_arr.shape[-1]))
    x = layers.Bidirectional(layers.GRU(256, return_sequences=True))(inputs)
    x = layers.Bidirectional(layers.GRU(256, return_sequences=True))(x)
    x = layers.Bidirectional(layers.GRU(256))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16)(x)
    outputs = layers.Dense(1, 'sigmoid')(x)

    model = keras.Model(inputs, outputs)
    
    model.summary()
    
    model_path = './GRU.h5'
    checkpointer = ModelCheckpoint(
        filepath=model_path,
        monitor='val_auc',
        verbose=1,
        mode='max',
        save_best_only=True)

    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['AUC'])
    
    return model, checkpointer

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Training
</h1>
</div>

In [ ]:
lstm_model, lstm_checkpointer = LSTM()
lstm_history = lstm_model.fit(x=x_train, y=y_train, validation_data = (x_val, y_val), batch_size=128, epochs=30, callbacks=[lstm_checkpointer])

In [ ]:
gru_model, gru_checkpointer = GRU()
gru_history = gru_model.fit(x=x_train, y=y_train, validation_data = (x_val, y_val), batch_size=128, epochs=30, callbacks=[gru_checkpointer])

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Evaluation on LSTM Model
</h1>
</div>

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = go.Figure()

labels = ['Loss', 'AUC']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 31),)*2)

y_data = np.array([
    np.array(lstm_history.history['loss']), 
                      np.array(lstm_history.history['auc'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[29],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[29]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LSTM Training Loss & AUC',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=500, width=800)

fig.show()

In [ ]:
fig = go.Figure()

labels = ['Val Loss', 'Val AUC']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 31),)*2)

y_data = np.array([
    np.array(lstm_history.history['val_loss']), 
                      np.array(lstm_history.history['val_auc'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[29],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[29]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='LSTM Validataion Loss & AUC',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=500, width=800)

fig.show()

In [ ]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve


model = load_model('./LSTM.h5')
y_pred = model.predict(x_val)
M = confusion_matrix(y_val, np.around(y_pred))

p = sns.heatmap(M, annot=True)
p.set_title("LSTM Model ROC Score : {:.2f}".format(roc_auc_score(y_val, np.around(y_pred))*100))

In [ ]:
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC-AUC Curve, Score : {:.2f}'.format(roc_auc_score(y_val, np.around(y_pred))*100))
    plt.legend()
    plt.show()
    
fper, tper, thresholds = roc_curve(y_val, np.around(y_pred))
plot_roc_curve(fper, tper)

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Evaluation on GRU Model
</h1>
</div>

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = go.Figure()

labels = ['Loss', 'AUC']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 31),)*2)

y_data = np.array([
    np.array(gru_history.history['loss']), 
                      np.array(gru_history.history['auc'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[29],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[29]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='GRU Training Loss & AUC',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=500, width=800)

fig.show()

In [ ]:
fig = go.Figure()

labels = ['Val Loss', 'Val AUC']
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']

mode_size = [8, 8]
line_size = [2, 2]

x_data = np.vstack((np.arange(1, 31),)*2)

y_data = np.array([
    np.array(gru_history.history['val_loss']), 
                      np.array(gru_history.history['val_auc'])
])

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {:.2f}'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[29],
                                  xanchor='left', yanchor='middle',
                                  text='{:.4f}%'.format(y_trace[29]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='GRU Validataion Loss & AUC',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Epochs',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,height=500, width=800)

fig.show()

In [ ]:
model = load_model('./GRU.h5')
y_pred = model.predict(x_val)
M = confusion_matrix(y_val, np.around(y_pred))

p = sns.heatmap(M, annot=True)
p.set_title("GRU Model ROC Score : {:.2f}".format(roc_auc_score(y_val, np.around(y_pred))*100))

In [ ]:
fper, tper, thresholds = roc_curve(y_val, np.around(y_pred))
plot_roc_curve(fper, tper)

## LSTM model works better than GRU!